# Lab 5: Agentic Document Extraction for RAG

In this lab, you will build a Retrieval-Augmented Generation (RAG) pipeline using ADE. RAG enables LLMs to answer questions about documents by retrieving relevant chunks and generating answers grounded in that context.

**Learning Objectives:**
- Understand the three phases of RAG: Preprocess, Retrieve, and Generate
- Set up a local ChromaDB vector database with OpenAI embeddings
- Incorporate visual grounding into queries for traceable information retrieval

## Background

Traditional keyword search fails when questions don't match exact document wording. RAG implements semantic search in three parts:
1. Preprocess: Parse documents into chunks, embed as vectors, store in database
2. Retrieve: Convert query to vector, find semantically similar chunks
3. Generate: Pass retrieved chunks as context to LLM for grounded answers

You will work with Apple's 10-K SEC filing (74 pages) pre-parsed using ADE (453 chunks). Each chunk includes text, bounding box coordinates, page number, and chunk type (eg text, table, figure).

## Outline

- [1. Import Libraries](#1)
- [2. Loading ADE-Parsed Data](#2)
- [3. Setting up the Vector Database](#3)
- [4. Querying the Database](#4)
- [5. Hybrid Search](#5)
- [6. RAG with LangChain](#6)

<a id="1"></a>

## 1. Import Libraries

Load libraries:

- **openai**: Generate vector embeddings (`text-embedding-3-small`)
- **chromadb**: Vector database for storing and querying embeddings
- **langchain**: Framework for building RAG chains

Additionally we have a script `helper.py` containing some utilities for cropping images and superimposing bounding boxes.  

In [ ]:
import os
import json
import re
from pathlib import Path
from dotenv import load_dotenv

from IPython.display import display, Image, IFrame, Markdown, JSON 

import helper

# OpenAI & ChromaDB - Embedding + Vector Store
import openai
import chromadb

# Langchain 
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Load environment variables from .env
_ = load_dotenv(override=True)

<a id="2"></a>

## 2. Loading ADE-Parsed Data

The Apple 10-K document has been pre-parsed using ADE. The output includes:
- Markdown: Full document converted to structured markdown with anchor tags
- JSON: Individual content chunks with metadata (bounding boxes, page numbers, chunk types)

This pre-computed output lets you focus on the RAG pipeline. In production, you would call the ADE Parse API directly.

In [ ]:
DOC_PATH = Path("apple_10k.pdf")
helper.print_document(DOC_PATH)

Set up paths to the pre-computed ADE outputs:

In [ ]:
# Setting Directories and Paths
OUTPUT_DIR = Path("./ade_outputs")
ADE_JSON_PATH = OUTPUT_DIR / "apple_10k_chunks.json"  
ADE_MD_PATH = OUTPUT_DIR / "apple_10k.md"

Preview the parsed markdown. Notice the anchor IDs which linking extracted information back to source locations.

In [ ]:
# Load and display markdown preview
print("\n Parsed Output (Page 1):")
with open(ADE_MD_PATH, "r", encoding="utf-8") as f:
    markdown_content = f.read()
    # Find first page content (up to first page break or 500 chars)
    first_page = markdown_content[:500]
    print(first_page + "...")

Load the JSON chunks. Each chunk contains:
- `chunk_id`: Unique identifier (matches anchor IDs in markdown)
- `chunk_type`: "text", "table", or "figure"
- `text`: The actual content
- `bbox`: Bounding box coordinates `[x0, y0, x1, y1]` (normalized 0-1)
- `page`: Page number (0-indexed)

In [ ]:
# Load the existing JSON chunks
with open(ADE_JSON_PATH, "r", encoding="utf-8") as f:
    loaded_chunks = json.load(f)

print(f"Loaded {len(loaded_chunks)} saved chunks.")

# Show first chunk structure
print(f"\n Sample chunk structure:")
print(json.dumps(loaded_chunks[0], indent=2)[:400] + "...")

print("\n Ready to query!")

<a id="3"></a>

## 3. Setting up the Vector Database

Set up ChromaDB to store vector embeddings. ChromaDB is a lightweight vector database that supports:
- **HNSW indexing**: Fast approximate nearest neighbor search
- **Metadata filtering**: Query by chunk type, page number, etc.
- **Persistence**: Data survives kernel restarts

You will use OpenAI's `text-embedding-3-small` to convert text into 1536-dimensional vectors.

In [ ]:
# Setting Directory and Collectioon name
CHROMA_DB_PATH = Path("./chroma_db")
COLLECTION_NAME = "ade_documents"

# embeding model for vector database 
EMBEDDING_MODEL = "text-embedding-3-small"

# Instantiate the Chroma Client
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)

# Create or Load ADE Collection
collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

Check if chunks are already in the database from a previous run:

In [ ]:
print(f"Checking for existing chunks in Chroma...")

# Get all existing chunk IDs from the collection
existing_result = collection.get(
                    ids=[chunk["chunk_id"] for chunk in loaded_chunks])
existing_ids = set(existing_result.get('ids', []))
print(f"Found {len(existing_ids)} existing chunks in collection")

For each new chunk, generate an embedding and store it with metadata:
- `chunk_type`: Filter by content type (text vs. table)
- `page`: Filter by page number
- `bbox_*`: Bounding box coordinates for visual grounding

In [ ]:
print(f"Inserting new chunks into Chroma...")

added_count = 0
for i, chunk in enumerate(loaded_chunks):
    chunk_id = chunk["chunk_id"]
    
    # Add chunk if it does not exist
    if chunk_id not in existing_ids:
        text = chunk.get("text", "")
        
        # Skip empty chunks
        if not text or not text.strip():
            continue

        # Generate Embeddings for chunk text with OpenAI
        emb = openai.embeddings.create(
            input=text,
            model=EMBEDDING_MODEL
        ).data[0].embedding
        
        # Flatten Metadata (Simple Types Only)
        metadata = {
            "chunk_type": chunk.get("chunk_type", "unknown"),
            "page": chunk.get("page", 0)
        }
        
        # Add bbox coordinates to metadata
        bbox = chunk.get("bbox")
        if bbox and len(bbox) == 4:
            metadata["bbox_x0"] = float(bbox[0])
            metadata["bbox_y0"] = float(bbox[1])
            metadata["bbox_x1"] = float(bbox[2])
            metadata["bbox_y1"] = float(bbox[3])
        
        # Store in Chroma
        collection.add(
            documents=[text],
            ids=[chunk_id],
            metadatas=[metadata],
            embeddings=[emb]
        )
        
        added_count += 1
        
        # Progress indicator
        if (added_count) % 20 == 0:
            print(f"   Processed {added_count} new chunks...")

print(f"\n✓ Added {added_count} new chunks, skipped existing chunks")

<a id="4"></a>

## 4. Querying the Database

Create a function to query the vector database:
1. Embed the query: Convert question to vector
2. Similarity search: Find chunks closest to query vector
3. Filter by threshold: Return only results above minimum similarity
4. Visual grounding: Display cropped images of source chunks

Similarity score is `1 - distance`, where distance is the L2 (Euclidean) distance between vectors.

In [ ]:
def rag_query(question, top_k=3, threshold=0.25, show_images=True):
    """
    Query the ADE Chroma index with a natural language question.
    Dynamically extracts and displays JUST the relevant chunk from 
    the PDF.
    
    Args:
        question (str): User query
        top_k (int): Max results to return
        threshold (float): Minimum similarity (1 - distance)
        show_images (bool): Display chunk-level grounding visualizations
    """
    # 1. Embed Query
    q_embed = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=question
    ).data[0].embedding

    # 2. Query Chroma
    results = collection.query(
        query_embeddings=[q_embed],
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )

    print(f"\n Query: {question}\n")
    print("=" * 80)

    # 3. Parse Results
    retrieved_docs = results["documents"][0]
    retrieved_meta = results["metadatas"][0]
    retrieved_dists = results["distances"][0]
    retrieved_ids = results["ids"][0]

    found_any = False
    for i, (text, meta, dist, cid) in enumerate(zip(
        retrieved_docs, retrieved_meta, retrieved_dists, retrieved_ids
    )):
        similarity = 1 - dist
         # Skip Weak Matches
        if similarity >= threshold:
            
            found_any = True
            page_num = meta.get('page', 0)
            chunk_type = meta.get('chunk_type', 'unknown')
            
            print(f"\n  Result {i+1} (similarity={similarity:.3f}):")
            print(f"   Chunk ID: {cid}")
            print(f"   Type: {chunk_type}, Page: {page_num}" )
            print(f"   Text preview: {text[:200]}...")
            
            # Display chunk-level grounding image
            if show_images:
                # Extract bbox from metadata if available
                bbox = None
                if all(k in meta for k in ['bbox_x0', 'bbox_y0', 'bbox_x1', 'bbox_y1']):
                    bbox = [
                        meta['bbox_x0'],
                        meta['bbox_y0'],
                        meta['bbox_x1'],
                        meta['bbox_y1']
                    ]
                
                # Dynamically extract chunk image from PDF
                print(f"\n Dynamically extracting chunk from PDF...")
                chunk_img = helper.extract_chunk_image(
                    pdf_path=DOC_PATH,
                    page_num=page_num,
                    bbox=bbox,
                    highlight=True,
                    padding=10
                )
                
                if chunk_img:
                    print(f"{chunk_type.title()} chunk (cropped):")
                    display(Image(data=chunk_img))
                else:
                    print(f"Could not extract chunk image")
          
        
        print("-" * 80)

    if not found_any:
        print("No results above similarity threshold.")
    
    return results


print("RAG query function defined with dynamic chunk extraction")

Test the query function. Visual grounding images verify the retrieved chunks contain relevant information:

In [ ]:
# Pass in Question in Natural Language, Top_K, and Threshold/L2 Distance
rag_query("What was Apple’s net sales in 2023?", 
          top_k=5, 
          threshold=0.32)

<a id="5"></a>

## 5. Hybrid Search

Semantic search retrieves chunks according to similarity. Hybrid search adds metadata filtering using the `where` parameter.

For example, when asking about revenue figures, search only tables since financial data is typically tabular. This avoids retrieving narrative text that mentions revenue in passing.

In [ ]:
q_embed = openai.embeddings.create(
    model=EMBEDDING_MODEL,
    input="What was Apple’s total revenue in 2023?",
).data[0].embedding

results = collection.query(
    query_embeddings=[q_embed],
    n_results=5,
    include=["documents", "metadatas", "distances"],
    where = {"chunk_type": "table"},
)

results["documents"]

<a id="6"></a>

## 6. RAG 

Combine retrieval with generation using LangChain. The `create_retrieval_chain` builds a pipeline that:
1. Takes a user question
2. Retrieves relevant chunks from the vector store
3. Formats them as context for the LLM
4. Generates a grounded answer

This abstracts the manual embedding and querying steps into a reusable chain.

First, wrap the ChromaDB collection with LangChain's `Chroma` class to get a retriever interface:

In [ ]:
vectordb = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=OpenAIEmbeddings(model = EMBEDDING_MODEL),
    persist_directory=str(CHROMA_DB_PATH)
)

retriever = vectordb.as_retriever()

Create the RAG chain with a prompt template. The `{context}` placeholder will be filled with retrieved chunks:

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their non-deterministic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [ ]:
# Define prompt template
system_prompt = (
    "Use the following pieces of retrieved context to answer the "
    "user's question. "
    "If you don't know the answer, say that you don't know."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Initialize LLM
llm = ChatOpenAI(model="gpt-5-mini", temperature = 1)

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, prompt | llm)

# Invoke the chain (conceptual)
response = rag_chain.invoke({"input": 
                             "What were Apple net sales in 2023"})
print(response["answer"])


Try a question requiring understanding trends across years:

In [ ]:
# Invoke the chain (conceptual)
response = rag_chain.invoke({
    "input": "How did total revenue trend between 2023 and 2022" 
                                       "for iPhone sales?"})
print(response["answer"])

## Summary

You built a complete RAG pipeline using ADE-parsed documents:

| Step | Component | Purpose |
|------|-----------|--------|
| **Parse** | ADE Parse API | Convert PDF to structured chunks with metadata |
| **Embed** | OpenAI `text-embedding-3-small` | Convert text to 1536-dim vectors |
| **Store** | ChromaDB | Persist vectors with metadata for fast retrieval |
| **Query** | Similarity Search | Find chunks semantically similar to query |
| **Retrieve** | Hybrid Search | Narrow results by chunk type, page, etc. |
| **Generate** | LangChain RAG Chain | Ground LLM answers in retrieved context |

Additionally, you learned to **verify** retrieved information and generated responses through visual grounding.

In our next lesson, we will learn to deploy ADE in the cloud — scaling our RAG application through an event-driven architecture that automatically triggers ADE for document processing whenever new documents appear. This lab helped us understand the logic that will drive our production-ready pipeline. Everything you learned here about embeddings, similarity search, threshold tuning, and metadata filtering transfers directly to Lesson 6. 